In [1]:
import polars as pl
import pandas as pd
import plotly.express as px 
import requests
from datetime import datetime
from pathlib import Path
from typing import List, Dict
import os

In [2]:
# constantes
MUSEO_URL="https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/4207def0-2ff7-41d5-9095-d42ae8207a5d/download/museos_datosabiertos.csv"
CINES_URL="https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/f7a8edb8-9208-41b0-8f19-d72811dcea97/download/salas_cine.csv"
ESPACIOS_URL="https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/01c6c048-dbeb-44e0-8efa-6944f73715d7/download/11_bibliotecapopular-datos-abiertos.csv"

list_dicts_url=[
 {
 "name":"museo", "url":MUSEO_URL    
 },
 {
 "name":"cines", "url":CINES_URL    
 },
 {
 "name":"espacios", "url":ESPACIOS_URL    
 }
]

# Path
BASE_FILE_DIR = Path("C:/Users/joan1/OneDrive/Documentos/Programacion/Python/Analisis de datos publicos ARG/tmp").resolve()

# tricks for iterate with a str path (Yuo can define in places holders)
file_path_crib = "{category}/{year}-{month:02d}/{category}-{day:02d}-{month:02d}-{year}.csv"

norm_cols= [
    "cod_localidad",
    "id_provincia",
    "id_departamento",
    "categoría",
    "provincia",
    "localidad",
    "nombre",
    "domicilio",
    "código postal",
    "número de teléfono",
    "mail",
    "web"
]

In [45]:
datetime.now().strftime("%Y-%m-%d")

'2023-10-08'

### Funtions

In [3]:
def get_response_and_create_csv(data_name:dict) -> pl.DataFrame:
    """
        Get the response from the url and create a csv file with the data
    """
    r=requests.get(
        url=data_name["url"]
    )
    r.encoding= "utf-8"

    now = datetime.now()
    category= data_name["name"]

    # create files and fill the places holders
    file_path = BASE_FILE_DIR.joinpath(file_path_crib.format(category=category, 
                                                             year=now.year, 
                                                             month=now.month, 
                                                             day=now.day))
    # create the file
    file_path.parent.mkdir(parents=True, exist_ok=True)

    with open(file=file_path, mode="w") as f:
        f.write(r.text)

    return pd.read_csv(file_path, encoding="latin-1").reset_index()

def columns_rename(df:pd.DataFrame, old_colums, new_columns):
    """
        Exchange the old columns names for the new columns names
    """
    list_changes = {old:new for old, new in zip(old_colums, new_columns)}
    print(list_changes)

    try:
        df = df.rename(columns=list_changes) 
    except Exception:
        print("The columns names not exist")
    return df


### Peticion get para obtener datos

In [4]:
# r=requests.get("https://datos.gob.ar/dataset/cultura-mapa-cultural-espacios-culturales/archivo/cultura_4207def0-2ff7-41d5-9095-d42ae8207a5d")
r=requests.get("https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/4207def0-2ff7-41d5-9095-d42ae8207a5d/download/museos_datosabiertos.csv")
r.encoding= "utf-8"

In [5]:
with open("open","w") as f_out: 
    f_out.write(r.text)

In [6]:
# pd.read_csv("open", delimiter=";", decimal=",",encoding="utf-8")
#pd.read_csv("open",encoding="utf-8-sig")
pd.read_csv("open",encoding="latin-1")

,Cod_Loc,IdProvincia,IdDepartamento,Observaciones,categoria,subcategoria,provincia,localidad,nombre,direccion,...,Mail,Web,Latitud,Longitud,TipoLatitudLongitud,Info_adicional,fuente,jurisdiccion,año_inauguracion,actualizacion
0,6588100,6,6588,NaN,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,9 de Julio,Archivo Y Museo Histórico Gral. Julio De Vedia,Libertad 1191,...,archivoymuseo@yahoo.com.ar,www.portaldel9.com.ar,-35.441762,-60.887598,Localización precisa,NaN,DNPyM,Municipal,1920.0,2017
1,6077010,6,6077,NaN,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,Arrecifes,Museo Y Archivo Histórico De Arrecifes,Gerardo Risso y España,...,museoarrecifes@hotmail.com,NaN,-34.066970,-60.102555,Localización precisa,NaN,DNPyM,Municipal,1972.0,2017
2,6035010,6,6035,NaN,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,Avellaneda,Museo Histórico-Sacro Hno. Rogelio Scortegagna,Calle 11 (Av. San Martín) Nº 830,...,NaN,NaN,-29.118024,-59.654543,Localización precisa,NaN,DNPyM,Municipal,NaN,2017
3,6035010,6,6035,NaN,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,Avellaneda,Museo Municipal De Artes Plásticas De Avellaneda,"Sarmiento 101, 1 º Piso",...,NaN,NaN,-34.662110,-58.363270,Localización precisa,NaN,DNPyM,Municipal,1956.0,2017
4,6049020,6,6049,NaN,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,Azul,Museo Etnográfico Y Archivo Histórico Enrique ...,Bartolomé J. Ronco 654,...,museoazul@yahoo.com,NaN,-36.781824,-59.858636,Localización precisa,NaN,DNPyM,Privada,1949.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,90098040,90,90098,NaN,Espacios de Exhibición Patrimonial,Museos,Tucumán,Tafí del Valle,Museo Histórico Y Arqueológico Jesuítico La Banda,La Banda S/N,...,NaN,NaN,-26.852880,-65.712622,Localización precisa,NaN,DNPyM,NaN,NaN,2017
1178,90105080,90,90105,NaN,Espacios de Exhibición Patrimonial,Museos,Tucumán,Tafí Viejo,Mac-Museo Arqueológico El Cadillal,Centro Principal el Cadillal,...,NaN,NaN,-26.683331,-65.266670,Localización precisa,NaN,DNPyM,Provincial,NaN,2017
1179,90105080,90,90105,NaN,Espacios de Exhibición Patrimonial,Museos,Tucumán,Tafí Viejo,Museo Ferroviario De Los Talleres De Tafí Viejo,Avda. Pte. Roque Sáenz Peña 234,...,NaN,NaN,-26.740733,-65.263231,Localización precisa,NaN,DNPyM,Privada,1909.0,2017
1180,90105070,90,90105,NaN,Espacios de Exhibición Patrimonial,Museos,Tucumán,Villa El Cadillal,Museo Arqueológico Dr. Ernesto Padilla,Ernesto Padilla S/N,...,NaN,NaN,-21.937642,-66.052043,Localización precisa,NaN,DNPyM,Provincial,NaN,2017


In [7]:
pl.read_csv("open", encoding= "latin-1")

Cod_Loc,IdProvincia,IdDepartamento,Observaciones,categoria,subcategoria,provincia,localidad,nombre,direccion,piso,CP,cod_area,telefono,Mail,Web,Latitud,Longitud,TipoLatitudLongitud,Info_adicional,fuente,jurisdiccion,año_inauguracion,actualizacion
i64,i64,i64,str,str,str,str,str,str,str,str,str,i64,str,str,str,f64,f64,str,str,str,str,i64,i64
6588100,6,6588,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""9 de Julio""","""Archivo Y Muse…","""Libertad 1191""",null,"""B6500EVL""",2317,"""425 279""","""archivoymuseo@…","""www.portaldel9…",-35.441762,-60.887598,"""Localización p…",null,"""DNPyM""","""Municipal""",1920,2017
6077010,6,6077,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Arrecifes""","""Museo Y Archiv…","""Gerardo Risso …",null,"""B2740FMJ""",2478,"""452 931""","""museoarrecifes…",null,-34.06697,-60.102555,"""Localización p…",null,"""DNPyM""","""Municipal""",1972,2017
6035010,6,6035,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Avellaneda""","""Museo Históric…","""Calle 11 (Av. …",null,"""S3561AKT""",3482,"""481 200""",null,null,-29.118024,-59.654543,"""Localización p…",null,"""DNPyM""","""Municipal""",null,2017
6035010,6,6035,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Avellaneda""","""Museo Municipa…","""Sarmiento 101,…",null,"""B1870CBC""",11,"""4205 9567""",null,null,-34.66211,-58.36327,"""Localización p…",null,"""DNPyM""","""Municipal""",1956,2017
6049020,6,6049,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Azul""","""Museo Etnográf…","""Bartolomé J. R…",null,"""B7300XAA""",2281,"""434 811""","""museoazul@yaho…",null,-36.781824,-59.858636,"""Localización p…",null,"""DNPyM""","""Privada""",1949,2017
6056010,6,6056,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Bahía Blanca""","""Museo De Bella…","""Sarmiento 450""",null,"""B8000HQJ""",291,"""459 4006""","""mac@bblanca.co…",null,-38.7131,-62.2622,"""Localización p…",null,"""DNPyM""","""Municipal""",1935,2017
6056010,6,6056,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Bahía Blanca""","""Museo De Cienc…","""Castelli 3702""",null,"""B8003APV""",291,"""488 3266""",null,null,-38.6928,-62.3087,"""Localización p…",null,"""DNPyM""","""Municipal""",1996,2017
6056010,6,6056,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Bahía Blanca""","""Museo Y Parque…","""Italia 19""",null,"""B8000DKA""",291,"""451 0675""",null,null,-38.724527,-62.273735,"""Localización p…",null,"""DNPyM""","""Privada""",2006,2017
6056010,6,6056,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Bahía Blanca""","""Museo De Arte …","""Sarmiento 450""",null,"""B8000HQJ""",291,"""459 4006""","""mac@bblanca.co…",null,-38.7131,-62.2622,"""Localización p…",null,"""DNPyM""","""Municipal""",2008,2017


### museo

In [8]:
df_museo = get_response_and_create_csv(list_dicts_url[0])

In [9]:
df_museo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1182 entries, 0 to 1181
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                1182 non-null   int64  
 1   Cod_Loc              1182 non-null   int64  
 2   IdProvincia          1182 non-null   int64  
 3   IdDepartamento       1182 non-null   int64  
 4   Observaciones        0 non-null      float64
 5   categoria            1182 non-null   object 
 6   subcategoria         1049 non-null   object 
 7   provincia            1182 non-null   object 
 8   localidad            1182 non-null   object 
 9   nombre               1182 non-null   object 
 10  direccion            1164 non-null   object 
 11  piso                 0 non-null      float64
 12  CP                   1116 non-null   object 
 13  cod_area             989 non-null    float64
 14  telefono             988 non-null    object 
 15  Mail                 264 non-null    o

In [10]:
df_museo.columns

Index(['index', 'Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Observaciones',
       'categoria', 'subcategoria', 'provincia', 'localidad', 'nombre',
       'direccion', 'piso', 'CP', 'cod_area', 'telefono', 'Mail', 'Web',
       'Latitud', 'Longitud', 'TipoLatitudLongitud', 'Info_adicional',
       'fuente', 'jurisdiccion', 'año_inauguracion', 'actualizacion'],
      dtype='object')

In [11]:
new_columns = [i for i in norm_cols if i not in df_museo.columns]
new_columns

['cod_localidad',
 'id_provincia',
 'id_departamento',
 'categoría',
 'domicilio',
 'código postal',
 'número de teléfono',
 'mail',
 'web']

In [12]:
old_columns = [
'Cod_Loc', 'IdProvincia',
 'IdDepartamento',
 'categoria',
 'direccion',
 'CP', 
 'telefono',
 'Mail', 
 'Web',
 ]

In [13]:
df_museo=columns_rename(df_museo, 
                        old_columns,
                        new_columns 
                        )
df_museo.columns

{'Cod_Loc': 'cod_localidad', 'IdProvincia': 'id_provincia', 'IdDepartamento': 'id_departamento', 'categoria': 'categoría', 'direccion': 'domicilio', 'CP': 'código postal', 'telefono': 'número de teléfono', 'Mail': 'mail', 'Web': 'web'}


Index(['index', 'cod_localidad', 'id_provincia', 'id_departamento',
       'Observaciones', 'categoría', 'subcategoria', 'provincia', 'localidad',
       'nombre', 'domicilio', 'piso', 'código postal', 'cod_area',
       'número de teléfono', 'mail', 'web', 'Latitud', 'Longitud',
       'TipoLatitudLongitud', 'Info_adicional', 'fuente', 'jurisdiccion',
       'año_inauguracion', 'actualizacion'],
      dtype='object')

In [14]:
df_museo[new_columns]

,cod_localidad,id_provincia,id_departamento,categoría,domicilio,código postal,número de teléfono,mail,web
0,6588100,6,6588,Espacios de Exhibición Patrimonial,Libertad 1191,B6500EVL,425 279,archivoymuseo@yahoo.com.ar,www.portaldel9.com.ar
1,6077010,6,6077,Espacios de Exhibición Patrimonial,Gerardo Risso y España,B2740FMJ,452 931,museoarrecifes@hotmail.com,NaN
2,6035010,6,6035,Espacios de Exhibición Patrimonial,Calle 11 (Av. San Martín) Nº 830,S3561AKT,481 200,NaN,NaN
3,6035010,6,6035,Espacios de Exhibición Patrimonial,"Sarmiento 101, 1 º Piso",B1870CBC,4205 9567,NaN,NaN
4,6049020,6,6049,Espacios de Exhibición Patrimonial,Bartolomé J. Ronco 654,B7300XAA,434 811,museoazul@yahoo.com,NaN
...,...,...,...,...,...,...,...,...,...
1177,90098040,90,90098,Espacios de Exhibición Patrimonial,La Banda S/N,4000,421 685,NaN,NaN
1178,90105080,90,90105,Espacios de Exhibición Patrimonial,Centro Principal el Cadillal,NaN,NaN,NaN,NaN
1179,90105080,90,90105,Espacios de Exhibición Patrimonial,Avda. Pte. Roque Sáenz Peña 234,4103,461 4335,NaN,NaN
1180,90105070,90,90105,Espacios de Exhibición Patrimonial,Ernesto Padilla S/N,4101,422 2199,NaN,NaN


### cines

In [15]:
df_cines = get_response_and_create_csv(list_dicts_url[1])

In [16]:
df_cines.sample(5)

,index,cod_localidad,id_provincia,id_departamento,categoria,provincia,departamento,localidad,nombre,direccion,...,latitud,longitud,tipo_latitud_longitud,fuente,sector,pantallas,butacas,tipo_de_gestion,espacio_incaa,año_actualizacion
56,56,6357110,6,6357,Salas de cine,Buenos Aires,General Pueyrredon,Mar del Plata,T.E.Y.C.I. Cinema,Rivadavia 3050,...,-37.997965,-57.552019,Precisa,INCAA / SInCA,Cine,2,487,Privada,No,2022
211,211,42105030,42,42105,Salas de cine,La Pampa,Maraco,Gral. Pico,Gran Pampa,Calle 20 642,...,-35.658982,-63.753935,Precisa,INCAA / SInCA,Cine,1,730,Privada,No,2022
295,295,82063170,82,82063,Salas de cine,Santa Fe,La Capital,Santa Fe,Cine America,25 De Mayo 3075,...,-31.640525,-60.704220,Precisa,INCAA / SInCA,Cine,1,380,Privada,No,2022
314,314,90119030,90,90119,Salas de cine,Tucumán,Yerba Buena,Yerba Buena,Solar del Cerro,Av. Aconquija 1336,...,-26.814179,-65.288937,Precisa,INCAA / SInCA,Cine,5,750,Privada,No,2022
116,116,2000010,2,2000,Salas de cine,Ciudad Autónoma de Buenos Aires,Ciudad Autonoma de Buenos Aires,Ciudad Autónoma de Buenos Aires,Atlas Caballito,Av. Rivadavia 5071,...,-34.618581,-58.437367,Precisa,INCAA / SInCA,Cine,9,1769,Privada,No,2022


In [17]:
df_cines.columns

Index(['index', 'cod_localidad', 'id_provincia', 'id_departamento',
       'categoria', 'provincia', 'departamento', 'localidad', 'nombre',
       'direccion', 'piso', 'cp', 'web', 'latitud', 'longitud',
       'tipo_latitud_longitud', 'fuente', 'sector', 'pantallas', 'butacas',
       'tipo_de_gestion', 'espacio_incaa', 'año_actualizacion'],
      dtype='object')

In [18]:
new_columns = [i for i in norm_cols if i not in df_cines.columns]
new_columns.remove("número de teléfono")
new_columns.remove("mail")
new_columns

['categoría', 'domicilio', 'código postal']

In [19]:

old_columns = ['categoria',
 'direccion',
 'cp',]

In [20]:
df_cines=columns_rename(df_cines, old_columns, new_columns)
df_cines[new_columns]

{'categoria': 'categoría', 'direccion': 'domicilio', 'cp': 'código postal'}


,categoría,domicilio,código postal
0,Salas de cine,Calle 9 entre 29 Y 30,6660
1,Salas de cine,Santa Rosa De Lima 170,6405
2,Salas de cine,Robbio 840,6500
3,Salas de cine,Hipolito Yrigoyen 13200,1846
4,Salas de cine,Segui 699,1846
...,...,...,...
311,Salas de cine,Av. Nestor Kirchner(Ex Av. Roca) 3450,4000
312,Salas de cine,San Martin 251,4000
313,Salas de cine,Av. Mitre 817,4000
314,Salas de cine,Av. Aconquija 1336,4107


### Espacios

In [21]:
df_espacios = get_response_and_create_csv(list_dicts_url[2])

In [22]:
new_columns = [i for i in norm_cols if i not in df_espacios.columns]
new_columns

['categoría', 'código postal', 'número de teléfono']

In [23]:
df_espacios.columns

Index(['index', 'cod_localidad', 'id_provincia', 'id_departamento',
       'observacion', 'categoria', 'subcategoria', 'provincia', 'departamento',
       'localidad', 'nombre', 'domicilio', 'piso', 'cp', 'cod_tel', 'telefono',
       'mail', 'web', 'informacion_adicional', 'latitud', 'longitud',
       'tipo_latitud_longitud', 'fuente', 'fecha_fundacion', 'nro_conabip',
       'anio_actualizacion'],
      dtype='object')

In [24]:
old_columns = ['categoria',
 'cp',
 'telefono',]
df_espacios=columns_rename(df_espacios, old_columns, new_columns)

{'categoria': 'categoría', 'cp': 'código postal', 'telefono': 'número de teléfono'}


In [25]:
df_espacios[new_columns]

,categoría,código postal,número de teléfono
0,Biblioteca Popular,6660,462234.0
1,Biblioteca Popular,6500,NaN
2,Biblioteca Popular,6500,426779.0
3,Biblioteca Popular,B6430CSL,430724.0
4,Biblioteca Popular,6441,432073.0
...,...,...,...
1897,Biblioteca Popular,4103,4619066.0
1898,Biblioteca Popular,4103,4461183.0
1899,Biblioteca Popular,4103,4610234.0
1900,Biblioteca Popular,4101,4370749.0


### Registros totales

In [26]:
list_df: List[pl.DataFrame]=[df_museo,df_cines,df_espacios]


In [27]:

list_colums = ["código postal", "número de teléfono", "mail", "web"]

In [28]:
df_all_culture_space=pd.concat([df_museo, df_cines, df_espacios],keys=["museos","cines","espacios"])

In [29]:
df_all_culture_space["one"]=1

#### por categoria, fuente y provincias

In [30]:
registros_by_category = df_all_culture_space.value_counts("categoría")
registros_by_category

categoría
Biblioteca Popular                    1902
Espacios de Exhibición Patrimonial    1182
Salas de cine                          316
Name: count, dtype: int64

In [31]:
registros_by_category.reset_index()

,categoría,count
0,Biblioteca Popular,1902
1,Espacios de Exhibición Patrimonial,1182
2,Salas de cine,316


In [32]:
registros_by_source = df_all_culture_space.value_counts("fuente")
registros_by_source

fuente
CONABIP                                                                1902
DNPyM                                                                  1048
INCAA / SInCA                                                           316
Municipalidad de Santa Fe - Red de Ciudades Creativas                    37
RCC- Córdoba                                                             31
Gobierno de la Provincia                                                 14
RCC                                                                      10
Área Gestión Cultural de la Municipalidad de Colonia Caroya               4
Red de Espacios Culturales Vicente López                                  4
Gobierno de la Provincia de Mendoza                                       4
Gobierno de la Provincia de Jujuy                                         4
Municipalidad de Río Gallegos - Red de Ciudades Creativas                 4
Gobierno de la Provincia de Chubut / SInCA 2013                           3
Gobie

In [41]:
registros_by_province = df_all_culture_space.groupby(["provincia","categoría"]).agg(
    count=("categoría","count") #*new_name_column = ("column_df map", "agg funtion")*
    )
registros_by_province.reset_index(inplace=True) # para que no quede la data like a groupby type, else like a dataframe
registros_by_province

,provincia,categoría,count
0,Buenos Aires,Biblioteca Popular,497
1,Buenos Aires,Espacios de Exhibición Patrimonial,152
2,Buenos Aires,Salas de cine,101
3,Catamarca,Biblioteca Popular,37
4,Catamarca,Espacios de Exhibición Patrimonial,32
...,...,...,...
67,Tierra del Fuego,Salas de cine,3
68,"Tierra del Fuego, Antártida e Islas del Atlánt...",Espacios de Exhibición Patrimonial,14
69,Tucumán,Biblioteca Popular,38
70,Tucumán,Espacios de Exhibición Patrimonial,26


### registros del cine

In [34]:
# Cines por provincias

df_cines.value_counts("provincia")

provincia
Buenos Aires                       101
Ciudad Autónoma de Buenos Aires     37
Córdoba                             36
Santa Fe                            21
Mendoza                             17
Tucumán                             11
Entre Ríos                          11
Chubut                               8
Santa Cruz                           8
Salta                                7
Corrientes                           7
Neuquén                              7
La Rioja                             6
Río Negro                            6
La Pampa                             5
Misiones                             4
Jujuy                                4
San Juan                             4
Santiago del Estero                  4
San Luis                             3
Chaco                                3
Tierra del Fuego                     3
Catamarca                            2
Formosa                              1
Name: count, dtype: int64

In [35]:
# cantidad de pantallas
df_cines.value_counts("pantallas")


pantallas
1     152
2      50
3      24
6      17
4      16
5      14
8      11
10      9
7       8
9       6
12      3
14      2
16      2
13      1
17      1
Name: count, dtype: int64

In [36]:
# cantidad de butacas
df_cines.value_counts("butacas")

butacas
350     7
300     7
400     5
220     4
500     4
       ..
419     1
430     1
431     1
432     1
4214    1
Name: count, Length: 257, dtype: int64

In [37]:
# cantidad de espacios incaa
df_cines.value_counts("espacio_incaa")

espacio_incaa
No    266
Si     50
Name: count, dtype: int64

In [38]:
df_cines['espacio_incaa']

0      No
1      No
2      No
3      No
4      No
       ..
311    No
312    Si
313    No
314    No
315    No
Name: espacio_incaa, Length: 316, dtype: object

In [39]:
df_cines.columns

Index(['index', 'cod_localidad', 'id_provincia', 'id_departamento',
       'categoría', 'provincia', 'departamento', 'localidad', 'nombre',
       'domicilio', 'piso', 'código postal', 'web', 'latitud', 'longitud',
       'tipo_latitud_longitud', 'fuente', 'sector', 'pantallas', 'butacas',
       'tipo_de_gestion', 'espacio_incaa', 'año_actualizacion'],
      dtype='object')

### registros totales por provincia

In [40]:
count_si= lambda x: sum(x=="Si")
df_cines.groupby("provincia", as_index=False).agg(
    cines = pd.NamedAgg(column="provincia", aggfunc="count"),
    pantallas = pd.NamedAgg(column="pantallas", aggfunc="sum"),
    butacas = pd.NamedAgg(column="butacas", aggfunc="sum"),
    espacio_incaa = pd.NamedAgg(column="espacio_incaa", aggfunc=count_si),
     )
#.agg({"pantallas":"sum", "butacas":"sum", "espacio_incaa":count_si})


,provincia,cines,pantallas,butacas,espacio_incaa
0,Buenos Aires,101,362,92818,18
1,Catamarca,2,10,1657,0
2,Chaco,3,13,2302,0
3,Chubut,8,9,2602,3
4,Ciudad Autónoma de Buenos Aires,37,150,27940,1
5,Corrientes,7,16,3170,0
6,Córdoba,36,110,22334,2
7,Entre Ríos,11,14,3842,1
8,Formosa,1,3,619,0
9,Jujuy,4,5,2277,2
